In [7]:
import requests
import pandas as pd

# API URL
url = "https://site.web.api.espn.com/apis/common/v3/sports/basketball/nba/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=true&page=1&limit=350&sort=offensive.avgPoints:desc"

# Fetch the data from the API
response = requests.get(url)
data = response.json()

# Process the data
athletes = data['athletes']

category_map = {}

for category in data['categories']:
    category_map[category['name']] = category['displayNames']

athlete_map = {}

for athlete in athletes:
    athlete_stats = {}
    # Assuming 'position' is directly available under 'athlete' and it has a property like 'abbreviation' for the position short form
    position = athlete['athlete'].get('position', {}).get('abbreviation', 'N/A')  # Use 'N/A' or similar if position is not available
    athlete_map[athlete['athlete']['displayName']] = athlete_stats
    athlete_stats['Position'] = position  # Add position to the athlete's stats
    for category in athlete['categories']:
        category_name = category['name']
        category_display_name = category_map.get(category_name, ['Unknown'])  # Fallback to 'Unknown' if not found
        for i in range(len(category['totals'])):
            athlete_stats[category_display_name[i]] = category['totals'][i]  # Ensure to index totals correctly
            
df = pd.DataFrame.from_dict(athlete_map, orient='index')


In [17]:
import pandas as pd
import boto3
from io import StringIO
from datetime import datetime

# Convert the DataFrame to a CSV string
csv_buffer = StringIO()
df.to_csv(csv_buffer)

# Get the current date to name the folder
date_str = datetime.now().strftime('%Y-%m-%d')
file_name = f"{date_str}/daily_stats.csv"

# Bucket name
bucket_name = 'nbagraphssamsean'

# Initialize a session using boto3
s3_resource = boto3.resource('s3')

# Upload the CSV file to S3
s3_resource.Object(bucket_name, file_name).put(Body=csv_buffer.getvalue())

print(f"File successfully uploaded to s3://{bucket_name}/{file_name}")


File successfully uploaded to s3://nbagraphssamsean/2024-02-28/daily_stats.csv


In [24]:
import matplotlib.pyplot as plt
import seaborn as sns
import io

# Assuming df is your DataFrame and it's already preprocessed
# Ensure that the FG% column is numeric. Sometimes scraping can bring in non-numeric types.
df['Field Goal Percentage'] = pd.to_numeric(df['Field Goal Percentage'], errors='coerce')

# Drop rows with NaN values in 'FG%' column that might have resulted from conversion
df.dropna(subset=['Field Goal Percentage'], inplace=True)

# Create the boxplot
plt.figure(figsize=(12, 10))
sns.boxplot(x='Position', y='Field Goal Percentage', data=df)

# Your mapping
position_names = {
    'PG': 'Point Guard',
    'SG': 'Shooting Guard',
    'SF': 'Small Forward',
    'PF': 'Power Forward',
    'C': 'Center',
    'F': 'Forward',
    'G': 'Guard'
}

# To fix the KeyError, we map the 'POS' column in df to their full names for plotting.
# This is done by creating a new column for display purposes.
df['Position Full Name'] = df['Position'].map(position_names)

# Recreate the boxplot with the updated DataFrame
sns.boxplot(x='Position Full Name', y='Field Goal Percentage', data=df)
plt.title('Boxplot of Field Goal Percentage by Position')
plt.xlabel('Position')
plt.ylabel('Field Goal Percentage')
plt.xticks(rotation=45)  # Optional: Rotate for better readability

# Instead of plt.show(), save the plot to a BytesIO buffer
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight')
buf.seek(0)

# Now, you can upload this buffer to S3 or another storage service
# Let's assume you want to upload it to S3.
# You would need the boto3 library and your AWS credentials set up for this.

s3 = boto3.client('s3')

object_name = f'{date_str}/field_goal_percentage_by_position.png'
s3.upload_fileobj(buf, bucket_name, object_name)

buf.close()  # Don't forget to close the buffer!

print(f"Plot successfully uploaded to s3://{bucket_name}/{object_name}")

Plot successfully uploaded to s3://nbagraphssamsean/2024-02-28/field_goal_percentage_by_position.png


In [25]:
df['3-Point Field Goal Percentage'] = pd.to_numeric(df['3-Point Field Goal Percentage'], errors='coerce')

# Drop rows with NaN values in 'FG%' column that might have resulted from conversion
df.dropna(subset=['3-Point Field Goal Percentage'], inplace=True)

# Create the boxplot
plt.figure(figsize=(12, 10))

sns.boxplot(x='Position Full Name', y='3-Point Field Goal Percentage', data=df)
plt.title('Boxplot of 3-Point Percentage by Position')
plt.xlabel('Position')
plt.ylabel('3-Point Percentage')
plt.xticks(rotation=45)  # Optional: Rotate for better readability

# Instead of plt.show(), save the plot to a BytesIO buffer
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight')
buf.seek(0)

# Now, you can upload this buffer to S3 or another storage service
# Let's assume you want to upload it to S3.
# You would need the boto3 library and your AWS credentials set up for this.

s3 = boto3.client('s3')

object_name = f'{date_str}/3point_percentage_by_position.png'
s3.upload_fileobj(buf, bucket_name, object_name)

buf.close()  # Don't forget to close the buffer!

print(f"Plot successfully uploaded to s3://{bucket_name}/{object_name}")

Plot successfully uploaded to s3://nbagraphssamsean/2024-02-28/3point_percentage_by_position.png


In [26]:
df['Free Throw Percentage'] = pd.to_numeric(df['Free Throw Percentage'], errors='coerce')

# Drop rows with NaN values in 'FG%' column that might have resulted from conversion
df.dropna(subset=['Free Throw Percentage'], inplace=True)

# Create the boxplot
plt.figure(figsize=(12, 10))

sns.boxplot(x='Position Full Name', y='Free Throw Percentage', data=df)
plt.title('Boxplot of Free Throw Percentage by Position')
plt.xlabel('Position')
plt.ylabel('Free Throw Percentage')
plt.xticks(rotation=45)  # Optional: Rotate for better readability

# Instead of plt.show(), save the plot to a BytesIO buffer
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight')
buf.seek(0)

# Now, you can upload this buffer to S3 or another storage service
# Let's assume you want to upload it to S3.
# You would need the boto3 library and your AWS credentials set up for this.

s3 = boto3.client('s3')

object_name = f'{date_str}/free_throw_percentage_by_position.png'
s3.upload_fileobj(buf, bucket_name, object_name)

buf.close()  # Don't forget to close the buffer!

print(f"Plot successfully uploaded to s3://{bucket_name}/{object_name}")

Plot successfully uploaded to s3://nbagraphssamsean/2024-02-28/free_throw_percentage_by_position.png
